In [11]:
import csv
import re
import spacy
from spacy import displacy
from owlready2 import *

# pre-processing
def Filter(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")
            
# stopwords from parsing the whole sentence
def RemoveStopword1(phrase, doc, chunkStart, chunkEnd, stopList):
    result = phrase
    i_stop=0
    #start = chunk.start# to eliminate the condition when the first word of chunk is stop word
    for i_sen in range(chunkStart, chunkEnd):
        while i_stop < len(stopList) and stopList[i_stop] < i_sen-1:
            #print(str(stopList[i_stop]) + ' ' + str(i_sen))
            i_stop = i_stop+1
        # there is no stop word in current chunk
        if i_stop >= len(stopList):
            break;
        #print(i_sen)
        # finish going through the chunk
        if stopList[i_stop] > chunk.end-1:
            break
        # find the stop word and remove it
        if stopList[i_stop] == i_sen-1:
            #print(doc[i_sen-1])
            if i_sen-1 == chunkStart:
                result = result.replace(doc[i_sen-1].text + ' ', '')
                chunkStart = chunkStart+1
            else:
                result = result.replace(' ' + doc[i_sen-1].text, '')
    return result

# stopwords from parsing triple separately
def RemoveStopword2(inputPhrase):
    result = ''
    doc_phrase = nlp(str(inputPhrase))
    for token in doc_phrase:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #       token.shape_, token.is_alpha, token.is_stop)
        if not token.is_stop:
            result = result + token.text + ' '
        #else:
        #    print(token.text + ', ', end = '')    
    return result

#load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)

senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array

file.close()
print("Total sentences: " + str(len(senSet)))

#pre-processing
Filter(senSet)

'''for sentence in senSet:
    print(senSet)'''

nlp = spacy.load('en_core_web_sm')

for index in range(len(senSet)):
    #index = 0
    
    # parse sentence
    doc = nlp(str(senSet[index]))
    print(str(index) + ': ' + senSet[index])

    # process result
    sub = ""
    pred = ""
    obj = ""
    stopList = []

    print('stop words: ', end='')
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #      token.shape_, token.is_alpha, token.is_stop)

        # record the index of stop words
        if token.is_stop:
            print(token.text + ', ', end='')
            stopList.append(token.i)
        if re.match('nsubj', token.dep_):   
            subj = token.text
        if re.match('ROOT', token.dep_): 
            pred = token.lemma_
            pred_orig = token.text
        if re.match('dobj', token.dep_): 
            obj = token.lemma_
    print('\n')

    subj_1 = subj
    obj_1 = obj
    # using chunk to update subject and object
    for chunk in doc.noun_chunks:
        if chunk.root.head.text == pred_orig and re.match('nsubj', chunk.root.dep_):
            subj = chunk.text
            # remove stop words
            subj_1 = RemoveStopword1(subj, doc, chunk.start, chunk.end, stopList)

        if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
            obj = chunk.text
            # remove stop words
            obj_1 = RemoveStopword1(obj, doc, chunk.start, chunk.end, stopList)
        #print(chunk.text + ' ' + str(chunk.start))
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

    print('Before : ' + subj + ' - ' + pred + ' - ' + obj)
    print('Method1: ' + subj_1 + ' - ' + pred + ' - ' + obj_1)

    # second method to remove stop words
    subj_2 = RemoveStopword2(subj)
    obj_2 = RemoveStopword2(obj)
    print('Method2: ' + subj_2 + '- ' + pred + ' - ' + obj_2 + '\n')

    ## visualize the semantic tree
    #options = {'compact': True, 'color': 'blue'}
    #displacy.serve(doc, style='dep', options=options)
    #displacy.serve(doc, style='dep')

Total sentences: 46
Pre-processing...
0: Sure um start off - I'm a Purdue grad
stop words: off, a, 

Before : I - be - a Purdue grad
Method1: I - be - Purdue grad
Method2: I - be - Purdue grad 

1:   Um my title is director of landscape architecture so um  for the ah staff of people
stop words: my, is, of, so, for, the, of, 

Before : my title - be - director
Method1: title - be - director
Method2: title - be - director 

2:   Um we manage all the construction and innovation projects for the university across the state
stop words: we, all, the, and, for, the, across, the, 

Before : we - manage - all the construction and innovation projects
Method1: we - manage - construction innovation projects
Method2: - manage - construction innovation projects 

3:    Some projects we handle in-house we design these others we're managing and consulting on your architecture 
stop words: we, in, we, these, others, we, and, on, your, 

Before : we - design - these others
Method1: we - design - others


Method2: - be - vision 

35:   Um so that can really be envisioned and that that's gonna take some work to get through that valley
stop words: so, that, can, really, be, and, that, that, take, some, to, get, through, that, 

Before : that - envision - work
Method1: that - envision - work
Method2: - envision - work 

36:   
stop words: 

Before : that -    - 
Method1: that -    - 
Method2: -    - 

37:   That's what we're hoping for
stop words: what, we, for, 

Before : we - be - 
Method1: we - be - 
Method2: - be - 

38:   And that would be you know those - a certain amount of just functionality with the zip line um in terms of you know it's gotta be open
stop words: that, would, be, you, those, a, amount, of, just, with, the, in, of, you, it, be, 

Before : it - get - those
Method1: it - get - those
Method2: - get - 

39:   You've got the cable above you
stop words: the, above, you, 

Before :   You - get - the cable
Method1:   You - get - cable
Method2:    You - get - cable 

40:   U

In [3]:
# https://pythonhosted.org/Owlready2/world.html
# https://stackoverflow.com/questions/24017320/using-owlclass-prefix-with-rdflib-and-xml-serialization
from owlready2 import *
import rdflib
#onto = get_ontology("dbpedia.owl").load()

my_world = World()
onto = my_world.get_ontology("dbpedia.owl").load()
my_world["http://test.org/onto/my_iri"]
#sync_reasoner(my_world)

graph = default_world.as_rdflib_graph()
r = list(graph.query("""SELECT ?p WHERE {
  <http://www.semanticweb.org/jiba/ontologies/2017/0/test#ma_pizza> <http://www.semanticweb.org/jiba/ontologies/2017/0/test#price> ?p .
}"""))
print(r)

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataPrope

[]
